## Training

In [1]:
import torchvision
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(5531)

# Prepare trainig data
train_data = torchvision.datasets.MNIST('./files/', 
                                  train=False, #Tak.
                                  download=True)

train_X = (torch.tensor(train_data.data)/255).to(torch.float)
train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1], train_X.shape[2])
train_y = train_data.targets

dataset = TensorDataset(torch.Tensor(train_X),torch.Tensor(train_y))
dataloader = DataLoader(dataset, batch_size=64)

print(train_X.shape)
print(train_y.shape)

torch.Size([10000, 1, 28, 28])
torch.Size([10000])


/var/folders/md/9788g9qs5h77d4hrzh39xpkc0000gn/T/ipykernel_2958/2281852813.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_X = (torch.tensor(train_data.data)/255).to(torch.float)


In [2]:
from methods.architectures import *

# Initialize and train classifiers
max_training_epochs = 250
training_support_level = 0.95

clfs = [
    CNN(architecure=CNN1_5_Network()),
    CNN(architecure=CNN1_10_Network()),
    CNN(architecure=CNN2_10_15_Network()),
    CNN(architecure=CNN1_20_Network()),
    CNN(architecure=CNN2_25_40_Network()),  
]

loss_fn = torch.nn.CrossEntropyLoss()

for clf_id, clf in enumerate(clfs):
    optimizer = torch.optim.SGD(clf.parameters(), lr=1e-2)

    for e in range(max_training_epochs):
        if e==0:
            clf.custom_train(dataloader, loss_fn, optimizer)
        else:

            proba = nn.Softmax(dim=1)( clf(train_X))
            max_proba = torch.max(proba, dim=1)[0] 
            mean_proba = torch.mean(max_proba).detach().numpy() # średnie wsparcie decyzyjne

            if mean_proba>training_support_level:
                print(clf_id, e, mean_proba)
                break
            
            clf.custom_train(dataloader, loss_fn, optimizer)
        
        if e%50==1:
            print(clf_id, e, mean_proba)
            

0 1 0.40806052
0 51 0.9354808
0 72 0.9502322
1 1 0.5474046
1 51 0.9404277
1 63 0.95050555
2 1 0.13053115
2 22 0.9500297
3 1 0.6263041
3 50 0.9502016
4 1 0.3133268
4 19 0.95116943
